In [ ]:
import requests
import time
from confluent_kafka import Producer
import json

producer_conf = {
    'bootstrap.servers': 'localhost:9092'  # Kafka server address
}

producer = Producer(producer_conf)

topic = 'test-topic'
# Function to handle delivery reports (optional)
def delivery_report(err, msg):
    if err is not None:
        print(f"Delivery failed for record {msg.key()}: {err}")
    else:
        print(f"Record successfully produced to {msg.topic()} [{msg.partition()}] at offset {msg.offset()}")

# Function to get data from the real-time API
def fetch_data_from_api():
    url = "https://dummyjson.com/quotes/random"
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()  # Return the data as a JSON object
    else:
        print(f"Error: Unable to fetch data (Status code: {response.status_code})")
        return None

try:
    while True:
        data = fetch_data_from_api()
        
        if data:
            message_value = json.dumps(data)
            
            producer.produce(topic, key=None, value=message_value, callback=delivery_report)
            
            producer.flush()
            
            print(f"Sent data to Kafka: {message_value}")
        
        time.sleep(5)

except KeyboardInterrupt:
    print("Producer interrupted")

finally:
    producer.flush()
    print("Finished streaming data.")

Record successfully produced to test-topic [0] at offset 15
Sent data to Kafka: {"id": 429, "quote": "It Is From Books That Wise People Derive Consolation In The Troubles Of Life.", "author": "Victor Hugo"}
Record successfully produced to test-topic [0] at offset 16
Sent data to Kafka: {"id": 681, "quote": "A Man Should Never Neglect His Family For Business.", "author": "Walt Disney"}
Record successfully produced to test-topic [0] at offset 17
Sent data to Kafka: {"id": 1422, "quote": "Coming Together Is A Beginning; Keeping Together Is Progress; Working Together Is Success.", "author": "Henry Ford"}
Record successfully produced to test-topic [0] at offset 18
Sent data to Kafka: {"id": 774, "quote": "You have to dream before your dreams can come true.", "author": "Abdul Kalam"}
Record successfully produced to test-topic [0] at offset 19
Sent data to Kafka: {"id": 1451, "quote": "Positive Thinking Will Let You Do Everything Better Than Negative Thinking Will.", "author": "Zig Ziglar"}
R